In [8]:
import pandas as pd
import numpy as np
import os
import nltk.classify.util
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.cross_validation import KFold, train_test_split
from BeautifulSoup import BeautifulSoup
from pymongo import MongoClient
from lxml import etree
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews

In [66]:
def word_feats(text):
    words = text.split(' ')
    return dict([(word, True) for word in words])

#Path of data files
pospath = r'C:\Anaconda\Galvanize\Minimester2-Project\txt_sentoken\pos'
negpath = r'C:\Anaconda\Galvanize\Minimester2-Project\txt_sentoken\neg'

#Other initializations
review = []
opinion = []
n_features = 2000
n_topics = 2
n_top_words = 20
pos_words = []
neg_words = []

#loop through neg files
for name in os.listdir(negpath):
    #context manager
    with open(negpath+'\\'+name, 'r') as fo:
        #read in the data
        data = fo.read()
        review.append(data.replace("'",""))
        opinion.append(False)

#loop through pos files
for name in os.listdir(pospath):
    #context manager
    with open(pospath+'\\'+name, 'r') as fo:
        #read in the data
        data = fo.read()
        review.append(data.replace("'",""))
        opinion.append(True)

negfeats = [(word_feats(review[f]), 'neg') for f in xrange(0,1000)]
posfeats = [(word_feats(review[f]), 'pos') for f in xrange(1000,2000)]

feats = np.vstack((np.array(negfeats),np.array(posfeats)))
reviews = np.array(review)
opinions = np.array(opinion)

feats_train, feats_test, reviews_train, reviews_test, opinions_train, opinions_test\
            = train_test_split(feats, reviews, opinions)

vectorizer = TfidfVectorizer(max_features=n_features, stop_words='english')

    
train_tfidf = vectorizer.fit_transform(list(reviews_train))
feature_names = vectorizer.get_feature_names()
test_tfidf = vectorizer.fit_transform(list(reviews_test))

nmf = NMF(n_components=n_topics,init='nndsvd').fit(train_tfidf)
z = nmf.transform(test_tfidf).dot(nmf.components_)

# for idx, Doc in enumerate(z):
#     if opinions_test[idx] == True:
#         pos_words.extend([feature_names[i] for i in Doc.argsort()[:-n_top_words - 1:-1]])
#     elif opinions_test[idx] == False:
#         neg_words.extend([feature_names[i] for i in Doc.argsort()[:-n_top_words - 1:-1]])
# #     break

# vectorizer2 = TfidfVectorizer(max_features=n_features, stop_words='english')

# pos_tfidf = vectorizer2.fit_transform(pos_words)
# neg_tfidf = vectorizer2.fit_transform(neg_words)
# logit_tfidf = np.vstack((pos_tfidf.todense(),neg_tfidf.todense()))

# pos = np.ones(pos_tfidf.shape[0])
# neg = np.zeros(neg_tfidf.shape[0])
# y = np.vstack((pos.reshape(-1,1),neg.reshape(-1,1)))

print z.shape
print opinions_test.shape

logit = LogisticRegression()
logit.fit_transform(z,opinions_test)
print logit.score(nmf.transform(train_tfidf).dot(nmf.components_),opinions_train)
yhat = logit.predict(nmf.transform(train_tfidf).dot(nmf.components_))
print yhat
print opinions_train

(500L, 2000L)
(500L,)
0.494
[False False False ..., False False False]
[False  True  True ..., False False  True]
